## 視覺化dataFrame(Table Visualization)
- 注意,在github上,無法顯示顏色
  

## 格式化值,index,column
- 必需install jinja2
- format()
- format_index()


In [98]:
import pandas as pd
import numpy as np
import matplotlib as mpl

df = pd.DataFrame({
    "strings": ["Adam", "Mike"],
    "ints": [1, 3],
    "floats": [1.123, 1000.23]
})
display(df)
s = df.style\
.format(precision=3, thousands=",", decimal=".")\
.format_index(str.upper, axis=1)\
.relabel_index(["row 1", "row 2"], axis=0)
print(type(s))
s 

,strings,ints,floats
0,Adam,1,1.123
1,Mike,3,1000.230


<class 'pandas.io.formats.style.Styler'>


,STRINGS,INTS,FLOATS
row 1,Adam,1,1.123
row 2,Mike,3,"1,000.230"


In [99]:
#綜合操作
weather_df = pd.DataFrame(np.random.rand(10,2)*5,
                        index=pd.date_range(start="2021-01-01", periods=10),
                        columns=["Tokyo", "Taipei"])

display(weather_df)

,Tokyo,Taipei
2021-01-01,2.200807,2.537057
2021-01-02,0.940155,2.039576
2021-01-03,1.662192,2.580890
2021-01-04,3.978916,1.268418
2021-01-05,3.593881,1.861095
2021-01-06,0.618895,3.593440
2021-01-07,1.747112,0.770645
2021-01-08,4.309515,4.757841
2021-01-09,4.391122,4.955585
2021-01-10,0.725009,0.988070


In [100]:
weather_df = pd.DataFrame(np.random.rand(10,2)*5,
                        index=pd.date_range(start="2021-01-01", periods=10),
                        columns=["Tokyo", "Taipei"])

def rain_condition(v):
    if v < 1.75:
        return "Dry"
    elif v < 2.75:
        return "Rain"
    return "Heavy Rain"

def make_pretty(styler):
    styler.set_caption("Weather Conditions")
    styler.format(rain_condition)
    styler.format_index(lambda v: v.strftime("%A"))
    styler.background_gradient(axis=None, vmin=1, vmax=5, cmap="PuBuGn")
    return styler

weather_df.loc["2021-01-04":"2021-01-08"].style.pipe(make_pretty)

,Tokyo,Taipei
Monday,Heavy Rain,Heavy Rain
Tuesday,Heavy Rain,Dry
Wednesday,Rain,Heavy Rain
Thursday,Dry,Heavy Rain
Friday,Rain,Dry


## 隱藏資料
- #### hide()

In [101]:
df = pd.DataFrame(np.random.randn(5,5))
display(df)

df.style \
.hide(subset=[0, 2, 4], axis=0)\
.hide(subset=[0, 2, 4], axis=1)

,0,1,2,3,4
0,0.575203,-1.218070,1.547686,-0.895299,0.443347
1,-1.286698,-0.309117,1.820968,-1.917814,0.916823
2,-2.030055,-0.002778,-1.554282,1.088606,-1.131921
3,0.719587,0.731870,0.366800,-1.695127,0.397308
4,-0.899006,1.218747,1.247380,0.510931,-0.923486


,1,3
1,-0.309117,-1.917814
3,0.731870,-1.695127


In [102]:
show = [0, 2, 4]
df.style\
.hide([row for row in df.index if row not in show], axis=0)\
.hide([col for col in df.columns if col not in show], axis=1)

,0,2,4
0,0.575203,1.547686,0.443347
2,-2.030055,-1.554282,-1.131921
4,-0.899006,1.247380,-0.923486


## 連結資料
- concat()

In [103]:
display(df.agg(["sum","mean"]))
summary_style = df.agg(["sum", "mean"]).style\
                                    .format(precision=3)\
                                    .relabel_index(["總分","平均"])
display(summary_style)
df.style.format(precision=3).concat(summary_style)

,0,1,2,3,4
sum,-2.920969,0.420652,3.428552,-2.908703,-0.297929
mean,-0.584194,0.084130,0.685710,-0.581741,-0.059586


,0,1,2,3,4
總分,-2.921,0.421,3.429,-2.909,-0.298
平均,-0.584,0.084,0.686,-0.582,-0.060


,0,1,2,3,4
0,0.575,-1.218,1.548,-0.895,0.443
1,-1.287,-0.309,1.821,-1.918,0.917
2,-2.030,-0.003,-1.554,1.089,-1.132
3,0.720,0.732,0.367,-1.695,0.397
4,-0.899,1.219,1.247,0.511,-0.923
總分,-2.921,0.421,3.429,-2.909,-0.298
平均,-0.584,0.084,0.686,-0.582,-0.060


In [104]:
## Styler支援HTML
df = pd.DataFrame([[38.0, 2.0, 18.0, 22.0, 21, np.nan],[19, 439, 6, 452, 226,232]],
                  index=pd.Index(['Tumour (Positive)', 'Non-Tumour (Negative)'], name='Actual Label:'),
                  columns=pd.MultiIndex.from_product([['Decision Tree', 'Regression', 'Random'],['Tumour', 'Non-Tumour']], names=['Model:', 'Predicted:']))
df.style

In [105]:
s = df.style.format('{:.0f}').hide([("Random","Tumour"),("Random","Non-Tumour")],axis="columns")
display(s)
print(s.to_html())

<style type="text/css">
</style>
<table id="T_2bc4b">
  <thead>
    <tr>
      <th class="index_name level0" >Model:</th>
      <th id="T_2bc4b_level0_col0" class="col_heading level0 col0" colspan="2">Decision Tree</th>
      <th id="T_2bc4b_level0_col2" class="col_heading level0 col2" colspan="2">Regression</th>
    </tr>
    <tr>
      <th class="index_name level1" >Predicted:</th>
      <th id="T_2bc4b_level1_col0" class="col_heading level1 col0" >Tumour</th>
      <th id="T_2bc4b_level1_col1" class="col_heading level1 col1" >Non-Tumour</th>
      <th id="T_2bc4b_level1_col2" class="col_heading level1 col2" >Tumour</th>
      <th id="T_2bc4b_level1_col3" class="col_heading level1 col3" >Non-Tumour</th>
    </tr>
    <tr>
      <th class="index_name level0" >Actual Label:</th>
      <th class="blank col0" >&nbsp;</th>
      <th class="blank col1" >&nbsp;</th>
      <th class="blank col2" >&nbsp;</th>
      <th class="blank col3" >&nbsp;</th>
    </tr>
  </thead>
  <tbody>
    <tr>
  

### 增加css的Styler
- #### set_table_style()
- #### set_td_classes()
- #### apply(), applymap(),apply_index(), applymap_index()

In [106]:
cell_hover = { # for row hover use <tr> instead of <td>
    'selector':'td:hover',
    'props':[('background-color','#ffffb3')]
}

index_names = {
    'selector':'.index_name',
    'props':'font-style:italic; color:darkgrey; font-weight:normal;'
}
headers = {
    'selector':'th:not(.index_name)',
    'props':'background-color:#000066; color:white;'
}
s.set_table_styles([cell_hover, index_names, headers])

In [107]:
s.set_table_styles([
    {'selector':'th.col_heading','props':'text-align:center;'},
    {'selector':'th.col_heading.level0','props':'font-size:1.5em'},
    {'selector':'td','props':'text-align:center; font-weight:bold;'}
],overwrite=False)
s

In [108]:
s.set_table_styles({
    ('Regression', 'Tumour'): [{'selector': 'th', 'props': 'border-left: 1px solid white'},
                               {'selector': 'td', 'props': 'border-left: 1px solid #000066'}]
}, overwrite=False, axis=0)

In [109]:
#set_td_classes()
s.set_table_styles([  # create internal CSS classes
    {'selector': '.true', 'props': 'background-color: #e6ffe6;'},
    {'selector': '.false', 'props': 'background-color: #ffe6e6;'},
], overwrite=False)
cell_color = pd.DataFrame([['true ', 'false ', 'true ', 'false '],
                           ['false ', 'true ', 'false ', 'true ']],
                          index=df.index,
                          columns=df.columns[:4])
s.set_td_classes(cell_color)